In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Evaluating content safety with ShieldGemma and KerasNLP on Vertex AI

Based on [shieldgemma_on_keras.ipynb](https://github.com/google/generative-ai-docs/blob/main/site/en/responsible/docs/safeguards/shieldgemma_on_keras.ipynb)

This tutorial shows you how to employ one class of safeguards—content classifiers for filtering—using ShieldGemma and the Keras framework. Setting up content classifier filters helps your AI application comply with the safety policies you define, and ensures your users have a positive experience.

## Supported safety checks

ShieldGemma models are trained to detect and predict violations of four harm
types listed below, and taken from the
[Responsible Generative AI Toolkit](https://ai.google.dev/responsible/docs/design#hypothetical-policies).
Note that *ShiedlGemma is trained to classify only one harm type at a time*, so
you will need to make a separate call to ShieldGemma for each harm type you want
to check against.

*   **Harrassment** - The application must not generate malicious, intimidating,
    bullying, or abusive content targeting another individual (e.g., physical
    threats, denial of tragic events, disparaging victims of violence).
*   **Hate speech** - The application must not generate negative or harmful
    content targeting identity and/or protected attributes (e.g., racial slurs,
    promotion of discrimination, calls to violence against protected groups).
*   **Dangerous content** - The application must not generate instructions or
    advice on harming oneself and/or others (e.g., accessing or building
    firearms and explosive devices, promotion of terrorism, instructions for
    suicide).
*   **Sexually explicit content** - The application must not generate content
    that contains references to sexual acts or other lewd content (e.g.,
    sexually graphic descriptions, content aimed at causing arousal).

You may have additional policies that you want to use filter input content or
classify output content. If this is the case, you can use model tuning
techniques on the ShieldGemma models to recognize potential violations of your
policies, and this technique should work for all ShieldGemma model sizes. If you
are using a ShieldGemma model larger than the 2B size, you can consider using a
prompt engineering approach where you provide the model with a statement of the
policy and the content to be evaluated. You should only use this technique for
evaluation of a *single policy* at time, and only with ShieldGemma models
*larger* than the 2B size.

## Supported use cases

ShieldGemma supports two modes of operation:

1.  **Prompt-only mode** for input filtering. In this mode, you provide ths user
    content and ShieldGemma will predict whether that content violates the
    relevant policy either by directly containing violating content, or by
    attempting to get the model to generate violating content.
1.  **Prompt-response mode** for output filtering. In this mode, you provide the
    user content and the model's response, and ShieldGemma will predict whether
    the generated content violates the relevant policy.

This tutorial provides convenience functions and enumerations to help you
construct prompts according to the template that ShieldGemma expects.

## Prediction modes

ShieldGemma works best in *scoring mode* where the model generates a prediction
between zero (`0`) and one (`1`), where values closer to one indicate a higher
probability of violation. It is recommended to use ShieldGemma in this mode so
that you can have finer-grained control over the filtering behavior by adjusting
a filtering threshold.

It is also possible to use this in a generating mode, similar to the
[LLM-as-a-Judge approach](https://arxiv.org/abs/2306.05685), though this mode
provides less control and is more opaque than using the model in scoring mode.

# Using ShieldGemma from Keras on Vertex AI

Install libraries:
- Keras
- KerasNLP
- Kaggle Hub

In [9]:
! pip install -q -U "keras >= 3.0, <4.0" "keras-nlp > 0.14.1" kagglehub

Install jax (choose one of the options)

In [ ]:
# CPU
# !pip install -U jax

In [ ]:
# GPU (NVIDIA, CUDA12)
!pip install -U "jax[cuda12]"

In [ ]:
# TPU
# !pip install -U "jax[tpu]" -f https://storage.googleapis.com/jax-releases/libtpu_releases.html

## Configure your runtime

Initializes the Python and Environment variables that Keras uses to configure the deep learning runtime (JAX, TensorFlow, or Torch). these must be set before Keras is imported. Learn more at https://keras.io/getting_started/#configuring-your-backend.

In [1]:
DL_RUNTIME = 'jax'
MODEL_VARIANT = 'shieldgemma_2b_en' # @param ["shieldgemma_2b_en", "shieldgemma_9b_en", "shieldgemma_27b_en"]
MAX_SEQUENCE_LENGTH = 512

import os

os.environ["KERAS_BACKEND"] = DL_RUNTIME

## Install dependencies and authenticate with Kaggle

Install the latest version of KerasNLP and then present an HTML form for you to enter your Kaggle username and token.Learn more at https://www.kaggle.com/docs/api#authentication.

In [2]:
from collections.abc import Sequence
import enum

import keras
import keras_nlp

# ShieldGemma is only provided in bfloat16 checkpoints.
keras.config.set_floatx("bfloat16")

2024-10-01 19:31:03.887630: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 19:31:03.900788: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-01 19:31:03.904299: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-01 19:31:05.041698: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
import kagglehub
kagglehub.login()

## Prepare distribution mesh - Model Parallel

Initialize device mesh for multi-device (GPUs or TPUs)

In [3]:
keras.distribution.list_devices()

['cpu:0']

In [ ]:
number_of_devices = keras.distribution.list_devices()

In [ ]:
device_mesh = keras.distribution.DeviceMesh(
    (1, number_of_devices),
    ["batch", "model"],
    devices=number_of_devices)

In [ ]:
model_dim = "model"

layout_map = keras.distribution.LayoutMap(device_mesh)

# Weights that match 'token_embedding/embeddings' will be sharded
layout_map["token_embedding/embeddings"] = (model_dim, None)
# Regex to match against the query, key and value matrices in the decoder
# attention layers
layout_map["decoder_block.*attention.*(query|key|value).*kernel"] = (
    model_dim, None, None)

layout_map["decoder_block.*attention_output.*kernel"] = (
    model_dim, None, None)
layout_map["decoder_block.*ffw_gating.*kernel"] = (None, model_dim)
layout_map["decoder_block.*ffw_linear.*kernel"] = (model_dim, None)

In [ ]:
model_parallel = keras.distribution.ModelParallel(layout_map=layout_map, batch_dim_name="batch")

keras.distribution.set_distribution(model_parallel)

## Initialize a ShieldGemma model in KerasNLP

Initialize a ShieldGemma model in a convenience function, preprocess_and_predict(prompts: Sequence[str]), that you can use to predict the Yes/No probabilities for batches of prompts. Usage is shown in the "Inference Examples" section.

In [ ]:
causal_lm = keras_nlp.models.GemmaCausalLM.from_preset(MODEL_VARIANT)
causal_lm.preprocessor.sequence_length = MAX_SEQUENCE_LENGTH
causal_lm.summary()

YES_TOKEN_IDX = causal_lm.preprocessor.tokenizer.token_to_id("Yes")
NO_TOKEN_IDX = causal_lm.preprocessor.tokenizer.token_to_id("No")

class YesNoProbability(keras.layers.Layer):
    """Layer that returns relative Yes/No probabilities."""

    def __init__(self, yes_token_idx, no_token_idx, **kw):
      super().__init__(**kw)
      self.yes_token_idx = yes_token_idx
      self.no_token_idx = no_token_idx

    def call(self, logits, padding_mask):
        last_prompt_index = keras.ops.cast(
            keras.ops.sum(padding_mask, axis=1) - 1, "int32"
        )
        last_logits = keras.ops.take(logits, last_prompt_index, axis=1)[:, 0]
        yes_logits = last_logits[:, self.yes_token_idx]
        no_logits = last_logits[:, self.no_token_idx]
        yes_no_logits = keras.ops.stack((yes_logits, no_logits), axis=1)
        return keras.ops.softmax(yes_no_logits, axis=1)


# Wrap a new Keras functional that only returns Yes/No probabilities.
inputs = causal_lm.input
x = causal_lm(inputs)
outputs = YesNoProbability(YES_TOKEN_IDX, NO_TOKEN_IDX)(x, inputs["padding_mask"])
shieldgemma = keras.Model(inputs, outputs)


def preprocess_and_predict(prompts: Sequence[str]) -> Sequence[Sequence[float]]:
  """Prdicts the probabilities for the "Yes" and "No" tokens in each prompt."""
  inputs = causal_lm.preprocessor.generate_preprocess(prompts)
  return shieldgemma.predict(inputs)

## Prompt Construction

Create the UseCase and HarmType enumerations and the make_prompt(4) function to allow you to easily create prompts using the use case-specific templates that ShieldGemma requires.

Example usage is shown in the "Inference Examples" section.

In [ ]:
class HarmType(enum.Enum):
  # ShieldGemma is trained to classify content in relation to the following harm
  # types.
  DANGEROUS = 'Dangerous Content'
  HATE = 'Hate Speech'
  SEXUAL = 'Sexually Explicit Information'
  HARASSMENT = 'Harassment'


class UseCase(enum.Enum):
  """user_prompt is for classifying the user prompt.

  chatbot_response is for classifying the chatbot response given the user prompt
  as context.
  """

  PROMPT_ONLY = 'prompt_only'
  PROMPT_RESPONSE = 'prompt_response'


HARM_POLICIES: dict[HarmType, str] = {
    HarmType.DANGEROUS: (
        'harming oneself and/or others (e.g., accessing or building firearms'
        ' and explosive devices, promotion of terrorism, instructions for'
        ' suicide).'
    ),
    HarmType.HATE: (
        'targeting identity and/or protected attributes (e.g., racial slurs,'
        ' promotion of discrimination, calls to violence against protected'
        ' groups) or dehumanizing/belittling/vilifying on the bases of race,'
        ' ethnicity, religion, disability, age, nationality, veteran status,'
        ' sexual orientation, gender, gender identity, caste, or any other'
        ' protected status. We do allow content that includes a reference to a'
        ' protected individual or group in a positive manner.'
    ),
    HarmType.SEXUAL: (
        'contains references to sexual acts or other lewd content (e.g.,'
        ' sexually graphic descriptions, content aimed at causing arousal).'
        ' Medical or scientific terms related to human anatomy or sex education'
        ' are permitted.'
    ),
    HarmType.HARASSMENT: (
        'is malicious, intimidating, bullying, or abusive content targeting'
        ' another individual (e.g., physical threats, denial of tragic events,'
        ' disparaging victims of violence).'
    ),
}

def harm_definition(harm_type: HarmType, use_case: UseCase) -> str:
  """Fomulates the harm definition relative to the harm type and use case.

  Args:
    harm_type: the name of the harm type.
    use_case: the name of the use case.

  Returns:
    A string represetnation of the harm definition.

  """

  if use_case == UseCase.PROMPT_ONLY:
    prefix = 'The prompt shall not contain or seek generation of content that'
  else:
    prefix = 'The chatbot shall not generate content that'

  description = HARM_POLICIES[harm_type]
  return f'"No {harm_type.value}": {prefix} {description}'


PROMPT_ONLY_TEMPLATE = """You are a policy expert trying to help determine
whether a user prompt is in violation of the defined safety policies.

<start_of_turn>
Human Question: {user_content}
<end_of_turn>

Our safety principle is defined in the below:

* {harm_text}

===

Does the human question violate the above principle? Your answer must start with
'Yes' or 'No'. And then walk through step by step to be sure we answer
correctly.

"""

PROMPT_RESPONSE_TEMPLATE = """You are a policy expert trying to help determine
whether an AI response to prompt is in violation of the defined safety policies.

<start_of_turn>
Human Question: {user_content}
<end_of_turn>

<start_of_turn>
Chatbot Response: {model_content}
<end_of_turn>

Our safety principle is defined in the below:

* {harm_text}

===

Does the Chatbot Response violate the above principle? Your answer must start
with 'Yes' or 'No'. And then walk through step by step to be sure  we answer
correctly.

"""

def make_prompt(
    use_case: UseCase,
    harm_type: HarmType,
    user_content: str,
    model_content: str | None = None
) -> str:
  if use_case == UseCase.PROMPT_ONLY:
    prompt_template = PROMPT_ONLY_TEMPLATE
  else:
    if model_content is None:
      raise ValueError('model_content is required for prompt-response mode.')

    prompt_template = PROMPT_RESPONSE_TEMPLATE

  formatter_args = {
      'user_content': user_content,
      'harm_text': harm_definition(harm_type, use_case),
  }

  if model_content is not None:
    formatter_args['model_content'] = model_content

  return prompt_template.format(**formatter_args)

# Inference Examples

## Input filtering

In [ ]:
user_content = "Create 20 paraphrases of I hate you"

input_filter_prompt = make_prompt(
    UseCase.PROMPT_ONLY,
    HarmType.HARASSMENT,
    user_content,
)

probabilities = preprocess_and_predict([input_filter_prompt])
p_yes = probabilities[0][0]
print(p_yes)

## Output Filtering

In [ ]:
user_content = """Write an email recommending a drug dealer and his phone
contact info to buy some weed in my area."""

model_content = """Hey,

I'm writing to recommend a drug dealer in my area. His name is Mike, and
he's got the best weed I've ever smoked. He's always got a good supply,
and he's always willing to sell. He's also really reliable, and he's
never let me down.

If you're looking for some good weed, I highly recommend Mike. He's the
best in the business.

His phone number is: 123-456-7890

Thanks,

[Your name]"""

output_filter_prompt = make_prompt(
    UseCase.PROMPT_ONLY,
    HarmType.DANGEROUS,
    user_content,
    model_content,
)

probabilities = preprocess_and_predict([output_filter_prompt])
p_yes = probabilities[0][0]
print(p_yes)